# iTEBD-4

In [1]:
import Tor10
import numpy as np
import copy

def Tprint(T):
    print(T.Print_diagram())
    print(T)

## Transfer Field Ising Model (TFIM)

$$
  H 
  = \sum_i -J \sigma^z_i \sigma^z_{i+1} - h \sigma^x_i
  = \sum_i -J \sigma^z_i \sigma^z_{i+1} - \frac{h}{2} \left( \sigma^x_i I_{i+1} + I_i \sigma^x_{i+1}\right)
  = \sum_i h_{i,i+1}.
$$
$$
  h_{i,i+1} = -J \sigma^z_i \sigma^z_{i+1} - \frac{h}{2} \left( \sigma^x_i I_{i+1} + I_i \sigma^x_{i+1}\right).
$$

In [2]:
# one-site operators
# Id, Sx, Sz
Id = Tor10.UniTensor(bonds=[Tor10.Bond(2), Tor10.Bond(2)], rowrank=1, name = 'Id')
Id.SetElem([1, 0,\
            0, 1])
Sx = Tor10.UniTensor(bonds=[Tor10.Bond(2), Tor10.Bond(2)], rowrank=1, name = 'Sx')
Sx.SetElem([0, 1,\
            1, 0])
Sz = Tor10.UniTensor(bonds=[Tor10.Bond(2), Tor10.Bond(2)], rowrank=1, name = 'Sz')
Sz.SetElem([1, 0,\
            0,-1])

print('Block form of Id, Sx, Sz:')
for T in [Id, Sx, Sz]:
    print(T.GetBlock())

# attach site index
Id1 = copy.deepcopy(Id)
Id2 = copy.deepcopy(Id)
Sx1 = copy.deepcopy(Sx)
Sx2 = copy.deepcopy(Sx)
Sz1 = copy.deepcopy(Sz)
Sz2 = copy.deepcopy(Sz)

# two-site operators
# Sz1⨂Sz2
Sz1.SetLabels([0, 2])
Sz2.SetLabels([1, 3])
Sz1_Sz2 = Tor10.Contract(Sz1, Sz2)
Sz1_Sz2.SetName('Sz1⨂Sz2')
# Sz1_Sz2.Print_diagram()
# print(Sz1_Sz2.GetBlock())

# Sx1⨂Id2
Sx1.SetLabels([0, 2])
Id2.SetLabels([1, 3])
Sx1_Id2 = Tor10.Contract(Sx1, Id2)
Sx1_Id2.SetName('Sx1⨂Id2')

# Id1⨂Sx2
Id1.SetLabels([0, 2])
Sx2.SetLabels([1, 3])
Id1_Sx2 = Tor10.Contract(Id1, Sx2)
Id1_Sx2.SetName('Id1⨂Sx2')

# Sz1⨂Id2
Sz1.SetLabels([0, 2])
Id2.SetLabels([1, 3])
Sz1_Id2 = Tor10.Contract(Sz1, Id2)
Sz1_Id2.SetName('Sz1⨂Id2')

# Id1⨂Sz2
Id1.SetLabels([0, 2])
Sz2.SetLabels([1, 3])
Id1_Sz2 = Tor10.Contract(Id1, Sz2)
Id1_Sz2.SetName('Id1⨂Sx2')

print('Block form of Sz1⨂Sz2, Sx1⨂Id2, Id1⨂Sx2, Sz1⨂Id2, Id1⨂Sz2:')
for T in [Sz1_Sz2, Sx1_Id2, Id1_Sx2, Sz1_Id2, Id1_Sz2]:
    print(T.GetBlock())

Block form of Id, Sx, Sz:
Tensor name: 
is_diag    : False
tensor([[1., 0.],
        [0., 1.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[0., 1.],
        [1., 0.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[ 1.,  0.],
        [ 0., -1.]], dtype=torch.float64)

Block form of Sz1⨂Sz2, Sx1⨂Id2, Id1⨂Sx2, Sz1⨂Id2, Id1⨂Sz2:
Tensor name: 
is_diag    : False
tensor([[ 1.,  0.,  0.,  0.],
        [ 0., -1.,  0., -0.],
        [ 0.,  0., -1., -0.],
        [ 0., -0., -0.,  1.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0., -1., -0.],
        [ 0.,  0

In [3]:
# set up parameters
J = 1.0
hx = 1.0
D = 20
dtau = 0.1

# set up two-site Hamiltonian
Ham_12 = Sz1_Sz2*(-J) - (Sx1_Id2+Id1_Sx2)*hx/2
Ham_12.SetName('Ham_12')

# set up exp(H_12*(-dtau))
expH = Tor10.ExpH(Ham_12.GetBlock()*(-dtau))
expH_12 = Tor10.UniTensor(bonds=Ham_12.bonds, rowrank=2, name='exp(H_12*(-dtau))')
expH_12.PutBlock(expH)

## Create MPS:
#
#     |    |     
#   --A-la-B-lb-- 
#
A = Tor10.UniTensor(bonds=[Tor10.Bond(D), Tor10.Bond(2), Tor10.Bond(D)], rowrank=1, labels=[-1,0,-2])
A.Rand()
A.SetName('A')
# A.Print_diagram()

B = Tor10.UniTensor(bonds=A.bonds,rowrank=1,labels=[-3,1,-4])
B.Rand()
B.SetName('B')
# B.Print_diagram()

la = Tor10.UniTensor(bonds=[Tor10.Bond(D),Tor10.Bond(D)], rowrank=1, labels=[-2,-3],is_diag=True)
la.Rand()
la.SetName('la')
# la.Print_diagram()

lb = Tor10.UniTensor(bonds=la.bonds, rowrank=1, labels=[-4,-5],is_diag=True)
lb.Rand()
lb.SetName('lb')
# lb.Print_diagram()

print("{:15s} {:15s} {:15s}".format('<E>','<Z>','<X>'))

# main iTEBD loop
for i in range(100):
    A.SetLabels([-1,0,-2])
    B.SetLabels([-3,1,-4])
    la.SetLabels([-2,-3])
    lb.SetLabels([-4,-5])

    ## X =
    #           (0)  (1)
    #            |    |     
    #  (-4) --lb-A-la-B-lb-- (-5) 
    #    
    
    X = Tor10.Contract(Tor10.Contract(A,la),Tor10.Contract(B,lb))
#     lb.Print_diagram()
#     lb.SetLabel(-1,idx=1)
    lb.SetLabels([-4,-1])
#     lb.Print_diagram()
    X = Tor10.Contract(lb,X)  
    X.SetName('X')
    
    # <MPS|MPS>
    XNorm = Tor10.Contract(X, X)

    # <MPS|Ham_12|MPS>    
    XH = Tor10.Contract(X, Ham_12)
    XH.SetName('XH')
    XH.SetLabels([-4,-5,0,1])
    XHX = Tor10.Contract(X, XH)

    # <MPS|Id1_Sz2|MPS>    
    Z2H = Tor10.Contract(X, Id1_Sz2)
    Z2H.SetName('Z2H')
    Z2H.SetLabels([-4,-5,0,1])
    XZ2X = Tor10.Contract(X, Z2H)

    # <MPS|Id1_Sx2|MPS>    
    X2H = Tor10.Contract(X, Id1_Sx2)
    X2H.SetName('Z2H')
    X2H.SetLabels([-4,-5,0,1])
    XX2X = Tor10.Contract(X, X2H)

    # measurements
    print("{:+.12f} {:+.12f} {:+.12f}".format(XHX.item()/XNorm.item(), XZ2X.item()/XNorm.item(), XX2X.item()/XNorm.item()))
    
    # imaginary time evolution
    XeH = Tor10.Contract(X,expH_12)
    XeH.SetName('XeH')
    XeH.Permute([-4,2,3,-5],rowrank=2,by_label=True)
    M = XeH.GetBlock()
    
    A,la,B = Tor10.Svd_truncate(M,D)
    la *= la.Norm()**-1 # re-normalize the norm of la
    AA = Tor10.UniTensor([XeH.bonds[0], XeH.bonds[1], A.bonds[1]], 
                         labels=[XeH.labels[0], XeH.labels[1], A.labels[1]], rowrank=2)
    AA.PutBlock(A)
    AA.SetRowRank(1)
    
    BB = Tor10.UniTensor([B.bonds[0], XeH.bonds[2], XeH.bonds[3]],
                        labels=[B.labels[0], XeH.labels[2], XeH.labels[3]], rowrank=1)
    BB.PutBlock(B)
    
    # de-contract the lb tensor , so it returns to 
    #             
    #            |     |     
    #       --lb-A'-la-B'-lb-- 
    #
    # again, but A' and B' are updated 

    lb_inv = Tor10.Inverse(lb)
    lb_inv.SetName('lb_inv')
    lb_inv.SetLabels([-5,-4])
    AA = Tor10.Contract(lb_inv, AA)
    BB = Tor10.Contract(BB, lb_inv)
    
    # translation symmetry, exchange A and B site
    A,B = BB,AA
    la,lb = lb,la

<E>             <Z>             <X>            
-0.958377622289 +0.141222123022 +0.967084349007
-0.980251829930 -0.033188329674 +0.983716792541
-1.113645946671 +0.082676473047 +0.969847226653
-1.093030368526 -0.009180997469 +0.945981855745
-1.169006459959 +0.064381236854 +0.924723516289
-1.150464738960 +0.008885171982 +0.901649355911
-1.194926473428 +0.055433220415 +0.881782522523
-1.181943012356 +0.020707927341 +0.862892865704
-1.208805336937 +0.050812513436 +0.846362558628
-1.200592697482 +0.028790219486 +0.831490712969
-1.217283528892 +0.048590011644 +0.818180250754
-1.212409240019 +0.034570578865 +0.806484347253
-1.223045332815 +0.047775593007 +0.795812176577
-1.220327410639 +0.038880158580 +0.786502713538
-1.227269895091 +0.047807002312 +0.777886309889
-1.225887060236 +0.042222336693 +0.770357219044
-1.230528573009 +0.048348051783 +0.763321375497
-1.229947374253 +0.044913160175 +0.757132735163
-1.233128221587 +0.049192020529 +0.751314166296
-1.233013573428 +0.047156066930 +0.74615